# Ollama test

In [16]:
import cv2
import ollama
from PIL import Image
import io

# ================= 配置区域 =================
MODEL_NAME = "qwen2.5vl:7b"  

# VIDEO_PATH = "./test_videos/video_24.mp4" 
VIDEO_PATH = "./output_L2CS/video_24.mp4" 
# VIDEO_PATH = "./output_gazelle/video_105.mp4" 

# 如果视频长 10 秒，SAMPLES=10 代表每秒截一张
TARGET_FRAME_COUNT = 20 
# ===========================================

def extract_frames(video_path, target_count=TARGET_FRAME_COUNT):
    """
    从视频中均匀抽取指定数量的帧，并转换为字节流
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 无法打开视频: {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    duration = total_frames / fps if fps > 0 else 0
    
    print(f"🎬 视频信息: 时长 {duration:.2f}s | 总帧数 {total_frames} | FPS {fps}")
    
    # 计算抽帧间隔
    step = max(1, total_frames // target_count)
    
    frame_images = [] 
    
    count = 0
    saved_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # 按照步长抽帧
        if count % step == 0 and saved_count < target_count:
            # OpenCV 默认是 BGR，需要转为 RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            
            # 为了加快速度和节省显存，可以将图片缩小，例如最大边长 720 或 512
            pil_img.thumbnail((960, 540)) 
            
            # 转为字节流
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='JPEG', quality=90)
            image_bytes = img_byte_arr.getvalue()
            
            frame_images.append(image_bytes)
            saved_count += 1
            
        count += 1

    cap.release()
    print(f"抽取 {len(frame_images)} 帧用于分析")
    return frame_images

def analyze_video(video_path):
    frames = extract_frames(video_path, target_count=TARGET_FRAME_COUNT)
    
    if not frames:
        return

    prompt = (
        f"You are an expert in micro-expression analysis. \n"
        # "You are analyzing a sequence of video frames marked with RED ARROWS indicating the gaze direction of the woman.\n\n"

        "Your task is to interpret a person’s body language."
        "You will be given a set of video frames representing a clip with one main speaker."
        "Follow the Task Guidelines."
        "Thinking out loud: <your thoughts about which interpretation is better>"

        "### STRICT ANALYSIS PIPELINE\n"
        "Follow these steps to decode the mental state:\n\n"

        "### Step 0: Fact\n"
        "- Given the fact: [She is smiling slightly.]\n"

        "1. **Vector Trajectory Tracing (The 'What')**:\n"
        "   - Compare the red arrow direction changes across timestamps.\n"
        "   - Is the arrow pointing at a fixed target ? OR is it pointing at different locations in each frame (e.g., Down -> Left -> Down-Right -> Straight)?\n"
        "   - **Key Question**: Define the 'Gaze Pattern'. Is it 'Stable/Focused' or 'Wandering/Unfocused'?\n\n"
        
        "2. **Incongruence Check (The 'Why')**:\n"
        "   - Observe her [facial expression] (Mouth/Cheeks). \n"
        "   - Contrast this with the 'Gaze Pattern' identified in Step 1. \n"
        "Is the expression consistent with the gaze behavior? or conflict?\n\n"
        
        "3. **Psychological Inference (The 'Meaning')**:\n"
        "   - Based on the specific [facial expression] + [Gaze Pattern] , what is her emotion?\n\n"
        
        "### Step 4: Final Inference\n"
        "- Based on the steps above, determine the mental state.\n"
        "- Choose between:\n"
        "   A. [Based on these observations, the speaker is likely not feeling joy.] \n"
        "   B. [Based on these observations, the speaker is likely feeling joy.] \n\n"
        
        "Output your reasoning for each step, and then provide the final conclusion."
    )

    # prompt = (     
    #     f"You are a socially intelligent body language expert. Your task is to interpret a person’s body language. "
    #     f"You will be given a set of video frames representing a clip with one main speaker. " 
    #     f"Follow the Task Guidelines and the Response Format.\n"            
    #     f"- You are given one fact about the speaker’s body language, and two possible interpretations.\n"
    #     f"- Think out loud about which interpretation is better given what you see in the images (2-3 sentences).\n"
    #     # add gaze
    #     f"- Please carefully observe the trajectory of the speaker's gaze (indicated by the red arrows) and eye blink in the whole video and think about it\n\n"
    #     f"- Finally, give your answer according to the Response Format.\n\n"
    #     f"Thinking out loud: <your thoughts about which interpretation is better (2-3 sentences)> \n"
    #     f"Answer: <A OR B>\n\n"
    #     f"### Fact: [She is smiling slightly.] \n\n"
    #     f"### Interpretations\n"
    #     f"A. [Based on these observations, the speaker is likely not feeling joy.] \n"
    #     f"B. [Based on these observations, the speaker is likely feeling joy.] \n\n"
    #     f"Please analyze the possible problems and give suggestions for improvement and rehabilitation actions."
    # )

    # prompt = ("describe what you have seen in the images in detail.")

    print("推理中")
    
    # 3. 发送给 Ollama:'images' 参数接受一个列表，我们把所有抽出来的帧都放进去
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{
                'role': 'user',
                'content': prompt,
                'images': frames  # 这里传入所有帧
            }],
                options={
                    'temperature': 0.1,
                    'num_ctx': 20000
                }
        )
        
        print("\n" + "="*20 + " 识别结果 " + "="*20)
        print(response['message']['content'])
        print("="*50)
        
    except Exception as e:
        print(f"❌ 推理出错: {e}")


analyze_video(VIDEO_PATH)

🎬 视频信息: 时长 3.52s | 总帧数 81 | FPS 23
抽取 20 帧用于分析
推理中

==================== 识别结果 ====================
### Step 0: Fact
- Given the fact: [She is smiling slightly.]

### Step 1: Vector Trajectory Tracing (The 'What')
- **Red Arrow Direction**: The red arrow in the frames points to the woman's face, indicating that she is the focus of attention.
- **Gaze Pattern**: The arrow remains consistently pointing to her face, suggesting a stable and focused gaze.

### Step 2: Incongruence Check (The 'Why')
- **Facial Expression**: The woman is smiling slightly. This is a positive expression, typically associated with happiness or contentment.
- **Incongruence**: The gaze pattern is stable and focused on her face, and the facial expression is consistent with a positive emotion. There is no incongruence between the gaze and the facial expression.

### Step 3: Psychological Inference (The 'Meaning')
- Based on the stable gaze and the positive facial expression, the woman appears to be feeling joy or co

# Ollama all dataset

In [1]:
import json
import os
import cv2
import io
import ollama
from PIL import Image

# ================= 配置区域 =================
MODEL_NAME = "qwen2.5vl:7b"  

VIDEO_FOLDER = "./videos"
# VIDEO_FOLDER = "./output_L2CS"
INPUT_FILE = "vibe_task.jsonl"
OUTPUT_FILE = "results_original_data+prompt_gaze.jsonl"

# 抽帧数量：根据显存大小调整
TARGET_FRAME_COUNT = 15
# ===========================================

def extract_frames(video_path, target_count=TARGET_FRAME_COUNT):
    """
    读取视频文件，均匀抽取指定数量的帧，并转为字节流列表
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 无法打开视频: {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # 简单的均匀抽帧逻辑
    step = max(1, total_frames // target_count)
    frame_images = []
    
    count = 0
    saved_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        if count % step == 0 and saved_count < target_count:
            # OpenCV BGR -> RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            
            # 缩放图片以节省显存 (可选，建议长边不超过 768 或 512)
            # Qwen2.5-VL 支持任意分辨率，但太大会导致推理慢
            pil_img.thumbnail((512, 512)) 
            
            # 转为字节流
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='JPEG', quality=70)
            frame_images.append(img_byte_arr.getvalue())
            saved_count += 1
            
        count += 1

    cap.release()
    return frame_images

# def process_data():
    # 检查输出文件是否存在，如果存在，统计行数以便断点续传（可选）
processed_ids = set()
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    data = json.loads(line)
                    processed_ids.add(data.get("task_id"))
                except:
                    pass

with open(INPUT_FILE, 'r', encoding='utf-8') as f_in, \
        open(OUTPUT_FILE, 'a', encoding='utf-8') as f_out: # 使用 'a' 模式追加
    
    for line_number, line in enumerate(f_in):
        if not line.strip(): continue
        
        item = json.loads(line)
        task_id = item.get("task_id")
        
        # 简单的断点续传逻辑
        if task_id in processed_ids:
            print(f"Skipping processed task: {task_id}")
            continue

        video_filename = item.get("video")
        fact_text = item.get("fact")
        inference_A = item.get("inference_A")
        inference_B = item.get("inference_B")
        
        # --- 步骤 1: 获取视频绝对路径 ---
        abs_video_path = os.path.abspath(os.path.join(VIDEO_FOLDER, video_filename))
        if not os.path.exists(abs_video_path):
            print(f"Error: Video file not found: {abs_video_path}")
            continue
        
        # --- 步骤 2: (核心修改) 抽取视频帧 ---
        # 本地 Ollama 不支持直接传文件路径，必须传图片列表
        print(f"Processing Task: {task_id} | Extracting frames from {video_filename}...")
        frames = extract_frames(abs_video_path, target_count=TARGET_FRAME_COUNT)
        
        if not frames:
            print(f"Error: No frames extracted for {task_id}")
            continue

        # --- 步骤 3: 构建 Prompt ---
        text_prompt = (     
            f"You are a socially intelligent body language expert. Your task is to interpret a person’s body language. "
            f"You will be given a set of video frames representing a clip with one main speaker. " 
            f"Strictly focus on the micro facial expressions and gaze motion."
            f"Follow the Task Guidelines and the Response Format.\n"            
            f"- You are given one fact about the speaker’s body language, and two possible interpretations.\n"
            f"- Think out loud about which interpretation is better given what you see in the images (2-3 sentences).\n"
            f"- Finally, give your answer according to the Response Format.\n\n"
            f"Thinking out loud: <your thoughts about which interpretation is better (2-3 sentences)> \n"
            f"Answer: <A OR B>\n\n"
            f"### Fact\n{fact_text}\n\n"
            f"### Interpretations\n"
            f"A. {inference_A}\n"
            f"B. {inference_B}\n\n"
            f"Please analyze the possible problems and give suggestions for improvement and rehabilitation actions."
        )

        # text_prompt = (
        #     f"You are an expert in micro-expression analysis. \n"
        #     # "You are analyzing a sequence of video frames marked with RED ARROWS indicating the gaze direction of the woman.\n\n"

        #     "Your task is to interpret a person’s body language."
        #     "You will be given a set of video frames representing a clip with one main speaker."
        #     "Follow the Task Guidelines."
        #     "Thinking out loud: <your thoughts about which interpretation is better>"

        #     "### STRICT ANALYSIS PIPELINE\n"
        #     "Follow these steps to decode the mental state:\n\n"

        #     "### Step 1: Fact\n"
        #     f"- Given the fact: [{fact_text}]\n"

        #     # "1. **Vector Trajectory Tracing (The 'What')**:\n"
        #     # "   - Compare the red arrow direction changes across timestamps.\n"
        #     # "   - Is the arrow pointing at a fixed target ? OR is it pointing at different locations in each frame (e.g., Down -> Left -> Down-Right -> Straight)?\n"
        #     # "   - **Key Question**: Define the 'Gaze Pattern'. Is it 'Stable/Focused' or 'Wandering/Unfocused'?\n\n"
            
        #     "2. **Incongruence Check (The 'Why')**:\n"
        #     "   - Observe her [facial expression] (Mouth/Cheeks). \n"
        #     # "   - Contrast this with the 'Gaze Pattern' identified in Step 1. \n"
        #     # "Is the expression consistent with the gaze behavior? or conflict?\n\n"
            
        #     "3. **Psychological Inference (The 'Meaning')**:\n"
        #     "   - Based on the specific [facial expression] , what is her emotion?\n\n"
            
        #     "### Step 4: Final Inference\n"
        #     "- Based on the steps above, determine the mental state.\n"
        #     "- Finally, give your ANSWER."
        #     "- Choose between:\n"
        #     f"   A. [{inference_A}] \n"
        #     f"   B. [{inference_B}] \n\n"

        #     "Output your reasoning for each step, and then provide the final conclusion."
        # )

        try:
            # --- 步骤 4: (核心修改) 调用 Ollama ---
            response = ollama.chat(
                model=MODEL_NAME,
                messages=[{
                    'role': 'user',
                    'content': text_prompt,
                    'images': frames  # 关键：传入抽取的帧数据列表
                }],
                options={
                    'temperature': 0.1,
                    # "num_batch": 256,
                    # 'num_ctx': 8192
                    # "num_gpu": 20
                }
            )
            
            response_content = response['message']['content']
            print(f"✅ Result for {task_id} generated.")

            # 保存结果
            result = {
                "task_id": task_id,
                "model_output": response_content,
                "ground_truth": item.get("answer"),
                "inference_A": inference_A, 
                "inference_B": inference_B
            }
            
            f_out.write(json.dumps(result, ensure_ascii=False) + "\n")
            f_out.flush()
            
        except Exception as e:
            print(f"❌ Error processing {task_id}: {e}")

Processing Task: video_1 | Extracting frames from video_1.mp4...
✅ Result for video_1 generated.
Processing Task: video_1 | Extracting frames from video_1.mp4...
✅ Result for video_1 generated.
Processing Task: video_2 | Extracting frames from video_2.mp4...
✅ Result for video_2 generated.
Processing Task: video_2 | Extracting frames from video_2.mp4...
✅ Result for video_2 generated.
Processing Task: video_2 | Extracting frames from video_2.mp4...
✅ Result for video_2 generated.
Processing Task: video_2 | Extracting frames from video_2.mp4...
✅ Result for video_2 generated.
Processing Task: video_3 | Extracting frames from video_3.mp4...
✅ Result for video_3 generated.
Processing Task: video_3 | Extracting frames from video_3.mp4...
✅ Result for video_3 generated.
Processing Task: video_3 | Extracting frames from video_3.mp4...
✅ Result for video_3 generated.
Processing Task: video_4 | Extracting frames from video_4.mp4...
✅ Result for video_4 generated.
Processing Task: video_5 | Ext

# 加载分割完的关键帧并推理整个数据集

In [ ]:
import json
import os
import glob
import io
import ollama
from PIL import Image

# ================= 配置区域 =================
MODEL_NAME = "qwen2.5vl:7b"  

# 关键帧保存的根目录
KEYFRAME_ROOT = "./keyframe_output"
INPUT_FILE = "vibe_task.jsonl"
OUTPUT_FILE = "results.jsonl"

MAX_IMAGE_WIDTH = 1080  # 最大宽度 (从 1920 降到 720)
JPEG_QUALITY = 80      # JPEG 质量 (降低质量减少体积)
# ===========================================

def load_keyframes_from_dir(task_id):
    """
    从关键帧目录加载已保存的关键帧图片
    """
    keyframe_dir = os.path.join(KEYFRAME_ROOT, task_id)
    
    if not os.path.exists(keyframe_dir):
        print(f"⚠️ 关键帧目录不存在: {keyframe_dir}")
        return []
    
    # 获取所有 jpg 文件并按名称排序
    jpg_files = sorted(glob.glob(os.path.join(keyframe_dir, "*.jpg")))
    
    if not jpg_files:
        print(f"⚠️ 在 {keyframe_dir} 中未找到 JPG 文件")
        return []
    
    frame_images = []
    
    for jpg_path in jpg_files:
        try:
            # with open(jpg_path, 'rb') as f:
            #     frame_data = f.read()
            #     frame_images.append(frame_data)
            with Image.open(jpg_path) as pil_img:
                # 1. 缩放到指定宽度（保持宽高比）
                pil_img.thumbnail((MAX_IMAGE_WIDTH, MAX_IMAGE_WIDTH), Image.Resampling.LANCZOS)
                
                # 2. 转为 JPEG 字节流（降低质量）
                img_byte_arr = io.BytesIO()
                pil_img.save(img_byte_arr, format='JPEG', quality=JPEG_QUALITY)
                frame_data = img_byte_arr.getvalue()
                frame_images.append(frame_data)
        except Exception as e:
            print(f"❌ 读取文件失败 {jpg_path}: {e}")
            continue
    
    print(f"✅ 从 {keyframe_dir} 加载了 {len(frame_images)} 张关键帧")
    return frame_images

# 断点续传逻辑
processed_ids = set()
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    data = json.loads(line)
                    processed_ids.add(data.get("task_id"))
                except:
                    pass

print(f"🔄 已处理任务数: {len(processed_ids)}")

with open(INPUT_FILE, 'r', encoding='utf-8') as f_in, \
        open(OUTPUT_FILE, 'a', encoding='utf-8') as f_out:
    
    for line in f_in:
        if not line.strip():
            continue
        
        item = json.loads(line)
        task_id = item.get("task_id")
        
        # 跳过已处理的任务
        if task_id in processed_ids:
            print(f"⏭️ 跳过已处理任务: {task_id}")
            continue
        
        print(f"\n🎬 处理任务: {task_id}")
        
        # === 核心修改：从关键帧目录加载图片 ===
        frames = load_keyframes_from_dir(task_id)
        
        if not frames:
            print(f"⚠️ 任务 {task_id} 无可用关键帧，跳过")
            continue
        
        fact_text = item.get("fact")
        inference_A = item.get("inference_A")
        inference_B = item.get("inference_B")
        
        text_prompt = (     
            f"You are a socially intelligent body language expert. Your task is to interpret a person’s body language, "
            # f"specifically looking for signs of masked emotions or social pretense. "  # [修改1: 明确任务是找伪装/掩饰]
            f"You will be given a sequence of keyframes capturing a specific gaze shift or eye movement of the speaker. " # [修改2: 告知VLM这些帧代表了眼神变化的过程]
            
            f"Strictly follow this analysis process:\n"
            f"1. Identify the overt facial expression (e.g., smiling, neutral).\n"
            f"2. Analyze the gaze motion across the frames.\n"
            f"3. Check for INCONGRUENCE: Does the gaze direction contradict the facial expression? \n\n" # [修改3: 强制进行不一致性检查，这是做对NT的关键]
            
            f"Follow the Task Guidelines and the Response Format.\n"            
            f"- You are given one fact about the speaker’s body language, and two possible interpretations.\n"
            f"- Think out loud about which interpretation is better given the gaze dynamics (2-3 sentences).\n"
            f"- Finally, give your answer according to the Response Format.\n\n"
            
            f"Thinking out loud: <your thoughts. Start by describing the gaze shift, then judge if it supports or contradicts the emotion shown by the mouth.>\n" # [修改4: 规定CoT的起手式]
            f"Answer: <A OR B>\n\n"
            f"### Fact\n{fact_text}\n\n"
            f"### Interpretations\n"
            f"A. {inference_A}\n"
            f"B. {inference_B}\n\n"
        )

        # text_prompt = (     
        #     f"You are a socially intelligent body language expert. Your task is to interpret a person’s body language. "
        #     f"You will be given a set of video frames representing a clip with one main speaker. " 
        #     f"Strictly focus on the micro facial expressions and gaze motion."
        #     f"Follow the Task Guidelines and the Response Format.\n"            
        #     f"- You are given one fact about the speaker’s body language, and two possible interpretations.\n"
        #     f"- Think out loud about which interpretation is better given what you see in the images (2-3 sentences).\n"
        #     f"- Finally, give your answer according to the Response Format.\n\n"
        #     f"Thinking out loud: <your thoughts about which interpretation is better (2-3 sentences)> \n"
        #     f"Answer: <A OR B>\n\n"
        #     f"### Fact\n{fact_text}\n\n"
        #     f"### Interpretations\n"
        #     f"A. {inference_A}\n"
        #     f"B. {inference_B}\n\n"
        #     f"Please analyze the possible problems and give suggestions for improvement and rehabilitation actions."
        # )
        
        try:
            print(f"🤖 发送给 VLM 推理中... (使用 {len(frames)} 张关键帧)")
            
            response = ollama.chat(
                model=MODEL_NAME,
                messages=[{
                    'role': 'user',
                    'content': text_prompt,
                    'images': frames  # 使用加载的关键帧
                }],
                options={
                    'temperature': 0.1,
                    'num_ctx': 48000
                }
            )
            
            response_content = response['message']['content']
            print(f"✅ 任务 {task_id} 推理完成")
            
            # 保存结果
            result = {
                "task_id": task_id,
                "model_output": response_content,
                "ground_truth": item.get("answer"),
                "inference_A": inference_A,
                "inference_B": inference_B,
            }
            
            f_out.write(json.dumps(result, ensure_ascii=False) + "\n")
            f_out.flush()
            
        except Exception as e:
            print(f"❌ 任务 {task_id} 推理失败: {e}")
            continue

print("\n🎉 所有任务处理完毕")

🔄 已处理任务数: 0

🎬 处理任务: video_1
✅ 从 ./keyframe_output/video_1 加载了 15 张关键帧
🤖 发送给 VLM 推理中... (使用 15 张关键帧)
✅ 任务 video_1 推理完成

🎬 处理任务: video_1
✅ 从 ./keyframe_output/video_1 加载了 15 张关键帧
🤖 发送给 VLM 推理中... (使用 15 张关键帧)
✅ 任务 video_1 推理完成

🎬 处理任务: video_2
✅ 从 ./keyframe_output/video_2 加载了 15 张关键帧
🤖 发送给 VLM 推理中... (使用 15 张关键帧)
✅ 任务 video_2 推理完成

🎬 处理任务: video_2
✅ 从 ./keyframe_output/video_2 加载了 15 张关键帧
🤖 发送给 VLM 推理中... (使用 15 张关键帧)
✅ 任务 video_2 推理完成

🎬 处理任务: video_2
✅ 从 ./keyframe_output/video_2 加载了 15 张关键帧
🤖 发送给 VLM 推理中... (使用 15 张关键帧)
✅ 任务 video_2 推理完成

🎬 处理任务: video_2
✅ 从 ./keyframe_output/video_2 加载了 15 张关键帧
🤖 发送给 VLM 推理中... (使用 15 张关键帧)
✅ 任务 video_2 推理完成

🎬 处理任务: video_3
✅ 从 ./keyframe_output/video_3 加载了 15 张关键帧
🤖 发送给 VLM 推理中... (使用 15 张关键帧)
✅ 任务 video_3 推理完成

🎬 处理任务: video_3
✅ 从 ./keyframe_output/video_3 加载了 15 张关键帧
🤖 发送给 VLM 推理中... (使用 15 张关键帧)
✅ 任务 video_3 推理完成

🎬 处理任务: video_3
✅ 从 ./keyframe_output/video_3 加载了 15 张关键帧
🤖 发送给 VLM 推理中... (使用 15 张关键帧)
✅ 任务 video_3 推理完成

🎬 处理任务: video_4
✅ 从 ./ke